# Init + Auth

In [1]:
import requests
from pprint import pprint

In [2]:
CLIENT_ID     = ""
CLIENT_SECRET = ""
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

if len( CLIENT_ID ) and len( CLIENT_SECRET ):
    print( "Authorization strings have been obtained!" )
else:
    print( "Auth info is MISSING!" )

Authorization strings have been obtained!


In [3]:
# POST
auth_response = requests.post( AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})
auth_response_data = auth_response.json()
access_token       = auth_response_data['access_token']

# Examples  
https://stmorse.github.io/journal/spotify-api.html

## Track Metadata

In [4]:
headers = { 'Authorization': 'Bearer {token}'.format(token=access_token) }
track_id = '6y0igZArWVi6Iz0rj35c1Y'
response = requests.get( BASE_URL + 'audio-features/' + track_id, headers = headers )
response = response.json()

In [5]:
pprint( response )

{'acousticness': 0.446,
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
 'danceability': 0.54,
 'duration_ms': 234910,
 'energy': 0.59,
 'id': '6y0igZArWVi6Iz0rj35c1Y',
 'instrumentalness': 0,
 'key': 0,
 'liveness': 0.14,
 'loudness': -4.359,
 'mode': 1,
 'speechiness': 0.0528,
 'tempo': 119.878,
 'time_signature': 4,
 'track_href': 'https://api.spotify.com/v1/tracks/6y0igZArWVi6Iz0rj35c1Y',
 'type': 'audio_features',
 'uri': 'spotify:track:6y0igZArWVi6Iz0rj35c1Y',
 'valence': 0.267}


## Get Albums by Artist

In [6]:
artist_id = '36QJpDe2go2KgaRleHCDTp'

# pull all artists albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()

for album in d['items']:
    print(album['name'], ' --- ', album['release_date'])

The Complete BBC Sessions (Remastered)  ---  2016-09-16
Celebration Day  ---  2012-11-19
Mothership (Remastered)  ---  2007
How the West Was Won (Remaster)  ---  2003-05-27
Coda (Remaster)  ---  1982-11-19
Coda (Deluxe Edition)  ---  1982-11-19
In Through the out Door (Remaster)  ---  1979-08-15
In Through the out Door (Deluxe Edition)  ---  1979-08-15
The Song Remains the Same (Remaster)  ---  1976-10-22
Presence (Remaster)  ---  1976-03-31
Presence (Deluxe Edition)  ---  1976-03-31
Physical Graffiti (Remaster)  ---  1975-02-24
Physical Graffiti (Deluxe Edition)  ---  1975-02-24
Houses of the Holy (Remaster)  ---  1973-03-28
Houses of the Holy (Deluxe Edition)  ---  1973-03-28
Led Zeppelin IV (Remaster)  ---  1971-11-08
Led Zeppelin IV (Deluxe Edition)  ---  1971-11-08
Led Zeppelin III (Deluxe Edition)  ---  1970-10-05
Led Zeppelin III (Remaster)  ---  1970
Led Zeppelin II (Remaster)  ---  1969-10-22
Led Zeppelin II (Deluxe Edition)  ---  1969-10-22
Led Zeppelin (Remaster)  ---  1969-

In [7]:
data = []   # will hold all track info
albums = [] # to keep track of duplicates

# loop over albums and get all tracks
for album in d['items']:
    album_name = album['name']

    # here's a hacky way to skip over albums we've already grabbed
    trim_name = album_name.split('(')[0].strip()
    if trim_name.upper() in albums or int(album['release_date'][:4]) > 1983:
        continue
    albums.append(trim_name.upper()) # use upper() to standardize
    
    # this takes a few seconds so let's keep track of progress    
    print(album_name)
    
    # pull all tracks from this album
    r = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks', 
        headers=headers)
    tracks = r.json()['items']
    
    for track in tracks:
        # get audio features (key, liveness, danceability, ...)
        f = requests.get(BASE_URL + 'audio-features/' + track['id'], 
            headers=headers)
        f = f.json()
        
        # combine with album info
        f.update({
            'track_name': track['name'],
            'album_name': album_name,
            'short_album_name': trim_name,
            'release_date': album['release_date'],
            'album_id': album['id']
        })
        
        data.append(f)

Coda (Remaster)
In Through the out Door (Remaster)
The Song Remains the Same (Remaster)
Presence (Remaster)
Physical Graffiti (Remaster)
Houses of the Holy (Remaster)
Led Zeppelin IV (Remaster)
Led Zeppelin III (Deluxe Edition)
Led Zeppelin II (Remaster)
Led Zeppelin (Remaster)
